# 1. Preparation
- All notebooks were tested on SageMaker Notebook only.

## Install package and change the local docker repo

In [1]:
install_needed = True
# install_needed = False

In [2]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    sudo service docker stop
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo rsync -aP /var/lib/docker /home/ec2-user/SageMaker/.container
    sudo service docker start
    echo "Docker Restart"
fi

# sudo curl -L "https://github.com/docker/compose/releases/download/v2.7.0/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
# sudo chmod +x /usr/local/bin/docker-compose

Redirecting to /bin/systemctl stop docker.service
  docker.socket


Add data-root and default-shm-size=10G
sending incremental file list
docker/
docker/engine-id
             36 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=36/38)
docker/buildkit/
docker/buildkit/cache.db
         32,768 100%    3.47MB/s    0:00:00 (xfr#2, to-chk=25/38)
docker/buildkit/containerdmeta.db
         16,384 100%    1.74MB/s    0:00:00 (xfr#3, to-chk=24/38)
docker/buildkit/history.db
         32,768 100%    3.47MB/s    0:00:00 (xfr#4, to-chk=23/38)
docker/buildkit/metadata_v2.db
         16,384 100%    1.74MB/s    0:00:00 (xfr#5, to-chk=22/38)
docker/buildkit/snapshots.db
         16,384 100%    1.74MB/s    0:00:00 (xfr#6, to-chk=21/38)
docker/buildkit/content/
docker/buildkit/content/ingest/
docker/buildkit/executor/
docker/containers/
docker/image/
docker/image/overlay2/
docker/image/overlay2/repositories.json
             19 100%    2.06kB/s    0:00:00 (xfr#7, to-chk=16/38)
docker/image/overlay2/distribution/
docker/image/overlay2/imagedb/
docker/image/overlay2/imagedb/conte

Redirecting to /bin/systemctl start docker.service


Docker Restart


In [3]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install --upgrade pip --quiet
    !{sys.executable} -m pip install -U sagemaker transformers datasets peft trl accelerate bitsandbytes --quiet
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel


If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it.



## Import packages and set configuration

In [1]:
import sagemaker
from pathlib import Path
from time import strftime

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

role = sagemaker.get_execution_role()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/04/25 21:36:51] INFO     Found credentials from IAM Role:                                   ]8;id=441520;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=123978;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


                    INFO     Found credentials from IAM Role:                                   ]8;id=467136;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=415923;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[05/04/25 21:36:52] INFO     Found credentials from IAM Role:                                   ]8;id=846924;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=452272;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [2]:
sagemaker.__version__

'2.243.3'

In [3]:
import os

os.environ['HF_DATASETS_CACHE'] = '/home/ec2-user/SageMaker/.cache'
os.environ['HF_CACHE_HOME'] = '/home/ec2-user/SageMaker/.cache'
os.environ['HUGGINGFACE_HUB_CACHE'] = '/home/ec2-user/SageMaker/.cache'
# os.environ['TRANSFORMERS_HOME'] = '/home/ec2-user/SageMaker/.cache'
# os.environ['HF_HOME'] = '/home/ec2-user/SageMaker/.cache'

## Download and upload a model and a tokenizer

In [4]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)
import torch
from datasets import load_dataset
import huggingface_hub
from trl import setup_chat_format

[05/04/25 21:36:54] INFO     PyTorch version 2.2.2 available.                                          ]8;id=144727;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/datasets/config.py\config.py]8;;\:]8;id=63371;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/datasets/config.py#54\54]8;;\

In [5]:
# test_model_id = 'Qwen/Qwen2.5-3B-Instruct'
test_model_id = 'Qwen/Qwen3-4B'

In [6]:
# huggingface_hub.login()

In [7]:
registered_model = test_model_id.split("/")[-1].lower().replace(".", "-")
print(f"registered_model : {registered_model}")
os.makedirs(registered_model, exist_ok=True)

huggingface_hub.snapshot_download(
    repo_id=test_model_id,
    revision="main",
    local_dir=registered_model
)

registered_model : qwen3-4b


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

'/home/ec2-user/SageMaker/TRAINING/qwen3-on-sagemaker/qwen3-4b'

In [8]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(test_model_id)
tokenizer.save_pretrained(f'./{registered_model}')

('./qwen3-4b/tokenizer_config.json',
 './qwen3-4b/special_tokens_map.json',
 './qwen3-4b/vocab.json',
 './qwen3-4b/merges.txt',
 './qwen3-4b/added_tokens.json',
 './qwen3-4b/tokenizer.json')

In [9]:
model_weight_path = sagemaker_session.upload_data(path=f'./{registered_model}', bucket=bucket, key_prefix=f"checkpoints/{registered_model}")
print('input spec (in this case, just an S3 path): {}'.format(model_weight_path))

input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-322537213286/checkpoints/qwen3-4b


In [10]:
hkcode_dataset = "llami-team/Korean-OpenThoughts-114k-Normalized"

In [11]:
test_yn = True  ## Test용으로 데이터셋 100개만 뽑아서 사용 + 예시 9개 추가
# test_yn = False ## 전체 데이터셋 사용 (오래 걸림)

In [12]:
split = "train[0:200]" if test_yn else "train"
    
dataset = load_dataset(
    hkcode_dataset,
    split=split,
    trust_remote_code=True,
)

In [13]:
additional_items = [
    {
        "question": "서울의 유명한 관광 코스에 대해 상세히 설명해 주세요.",
        "reasoning": "서울은 한국의 수도로서 다양한 관광 명소가 있다. 이 질문에 답하기 위해 서울의 주요 관광 코스를 카테고리별로 나누어 설명해야 한다. 역사/문화 코스, 현대적인 코스, 자연 코스, 트렌디한 코스, 전통시장 코스 등으로 구분할 수 있다. 각 코스별로 대표적인 장소 3곳 정도를 선정하고, 각 장소의 특징과 볼거리를 간략히 설명하는 것이 좋겠다. 또한 이동 방법이나 팁도 추가하면 유용할 것이다.",
        "response": "서울은 다양한 관광 명소와 매력적인 코스를 갖춘 도시입니다. 여기 서울의 대표적인 관광 코스들을 소개해 드리겠습니다:\n\n## 역사와 문화의 코스\n\n### 경복궁-북촌 한옥마을-인사동 코스\n\n- **경복궁**: 조선시대 최초의 궁궐로, 웅장한 건축물과 아름다운 정원을 감상할 수 있습니다.\n- **북촌 한옥마을**: 전통 한옥이 잘 보존된 마을로, 한복 체험, 전통 공방, 갤러리 등을 즐길 수 있습니다.\n- **인사동**: 전통 문화의 중심지로, 골동품 상점, 갤러리, 전통 찻집 등이 있습니다.\n\n## 현대적인 서울 코스\n\n### 명동-남산서울타워-동대문디자인플라자(DDP) 코스\n\n- **명동**: 쇼핑과 음식의 천국으로, 다양한 브랜드 매장과 길거리 음식을 즐길 수 있습니다.\n- **남산서울타워**: 서울의 상징적인 랜드마크로, 도시의 파노라마 뷰를 감상할 수 있습니다.\n- **DDP**: 독특한 건축 디자인으로 유명한 복합 문화 공간입니다.\n\n## 자연과 함께하는 코스\n\n### 한강공원-여의도-선유도공원 코스\n\n- **한강공원**: 자전거 타기, 피크닉, 강변 산책을 즐길 수 있습니다.\n- **여의도**: 금융 중심지이자 봄에는 벚꽃 축제로 유명합니다.\n- **선유도공원**: 폐정수장을 재활용한 생태공원으로, 독특한 분위기를 자랑합니다.\n\n## 트렌디한 서울 코스\n\n### 홍대-연남동-경의선숲길 코스\n\n- **홍대**: 젊은이들의 문화 중심지로, 다양한 카페, 바, 클럽, 독특한 상점들이 있습니다.\n- **연남동**: 아기자기한 카페와 레스토랑이 많은 힙한 동네입니다.\n- **경의선숲길**: 옛 철길을 공원으로 조성한 곳으로, 산책하기 좋습니다.\n\n## 전통시장 체험 코스\n\n### 광장시장-동대문시장-남대문시장 코스\n\n- **광장시장**: 다양한 전통 음식을 맛볼 수 있는 유서 깊은 시장입니다.\n- **동대문시장**: 24시간 운영되는 패션의 메카입니다.\n- **남대문시장**: 다양한 상품을 저렴하게 구매할 수 있는 대형 시장입니다.\n\n이 코스들은 서울의 다양한 면모를 경험할 수 있도록 구성되어 있습니다. 역사, 문화, 현대성, 자연, 쇼핑 등 서울의 모든 매력을 느낄 수 있을 것입니다. 각 코스는 대중교통으로 쉽게 이동할 수 있으며, 서울 시내 곳곳에 위치한 따릉이(공공자전거) 대여소를 이용하면 더욱 편리하게 여행할 수 있습니다."
    },
    {
        "question": "경상남도 고성의 유명한 관광 코스에 대해 상세히 설명해 주세요.",
        "reasoning": "경상남도 고성은 공룡 화석지로 유명한 지역이다. 이 질문에 답하기 위해서는 고성의 주요 관광지와 코스를 체계적으로 설명해야 한다. 특히 상족암 공룡 발자국 화석지와 관련 시설, 당항포 관광지, 그리고 기타 역사 문화 관광지를 포함해야 한다. 또한 고성에서 열리는 주요 축제와 이벤트도 언급하면 좋을 것이다. 각 관광지의 특징과 볼거리를 구체적으로 설명하고, 코스별로 정리하여 제시하는 것이 효과적이다.",
        "response": "경상남도 고성은 공룡 관련 관광지로 유명한 곳입니다. 주요 관광 코스는 다음과 같습니다:\n\n## 상족암 공룡 발자국 화석지 코스\n\n- **상족암 군립공원**: 세계 최대 규모의 공룡 발자국 화석지가 있는 곳입니다.\n- **고성공룡박물관**: 상족암 공원 내에 위치하며, 공룡 화석과 모형을 전시하고 있습니다.\n- **해안 산책로**: 공룡 발자국을 직접 볼 수 있는 목재 데크가 설치되어 있습니다.\n\n### 주요 특징\n- 약 6,000개의 공룡 발자국 화석이 발견되었습니다.\n- 420개의 공룡 보행렬이 있습니다.\n- 세계에서 유일하게 완벽한 피부 자국이 남아있는 공룡 발자국 4개가 발견되었습니다.\n\n## 당항포 관광지 코스\n\n- **당항포 관광지**: 경남고성공룡세계엑스포의 주요 개최지입니다.\n- **해양레저스포츠**: 다양한 해양 활동을 즐길 수 있습니다.\n- **펜션 및 오토캠핑장**: 숙박 시설이 마련되어 있습니다.\n\n## 기타 관광 명소\n\n- **고성박물관**: 고성의 역사와 문화재를 소개하는 박물관입니다.\n- **송학동 고분군**: 소가야 시대의 왕릉으로, 아름다운 경관으로 유명합니다.\n- **해개 해안 둘레길**: 남포항에서 남산 오토캠핑장까지 연결되는 1.4km 구간의 해안 산책로입니다.\n\n## 축제 및 이벤트\n\n- **경남고성공룡세계엑스포**: 9월과 10월에 개최되는 공룡 관련 축제입니다.\n- **고성 가리비 수산물 축제**: 10월에 열리는 해산물 축제입니다.\n- **고성 도루묵 축제**: 12월 초에 개최되며, 도루묵 낚시, 경매 등 다양한 행사가 열립니다.\n\n고성은 공룡 화석과 아름다운 해안 경관, 그리고 다양한 축제로 관광객들에게 인기 있는 목적지입니다. 역사, 자연, 그리고 현대적인 관광 시설이 잘 어우러진 관광 코스를 즐길 수 있습니다."
    },
    {
        "question": "전세계 미슐랭 3스타 식당의 개수는 몇 개인가요?",
        "reasoning": "이 질문에 답하기 위해서는 최신 미슐랭 가이드 정보를 바탕으로 전 세계 미슐랭 3스타 레스토랑의 수를 제공해야 한다. 2022년 9월 기준 내 지식으로는 약 135개 정도였으나, 이는 매년 변동되므로 2024년 기준으로 업데이트된 수치를 제공해야 한다. 또한 주요 국가별 대표적인 3스타 레스토랑 몇 곳을 예시로 들면 답변의 신뢰도를 높일 수 있다. 프랑스, 일본, 미국, 이탈리아 등 미슐랭 3스타 레스토랑이 많은 국가들의 유명 레스토랑을 언급하고, 한국의 상황도 포함하면 좋을 것이다.",
        "response": "2024년 기준으로 전 세계 미슐랭 3스타 레스토랑의 개수는 145개입니다. 주요 국가별 대표적인 3스타 레스토랑은 다음과 같습니다:\n\n프랑스\n프랑스는 미슐랭 3스타 레스토랑이 가장 많은 국가입니다.\nRestaurant Paul Bocuse (L'Auberge du Pont de Collonges): 세계에서 가장 오래된 3스타 레스토랑\nGuy Savoy (파리)\nL'Ambroisie (파리)\n\n일본\n일본은 프랑스 다음으로 3스타 레스토랑이 많은 국가입니다.\nSukiyabashi Jiro (도쿄)\nKanda (도쿄)\nKikunoi Honten (교토)\n\n미국\n미국에는 약 12개의 3스타 레스토랑이 있습니다.\nMasa (뉴욕): 다카야마 마사 셰프의 오마카세 레스토랑\nAlinea (시카고): 독특한 후각 경험을 제공하는 레스토랑\nThe French Laundry (캘리포니아)\n\n독일\nJAN (뮌헨): 2023년에 0에서 3스타로 직접 상승한 주목할 만한 레스토랑\n\n스페인\nArzak (산세바스티안)\nEl Celler de Can Roca (지로나)\n\n이탈리아\nOsteria Francescana (모데나)\nPiazza Duomo (알바)\n\n한국\n한국에는 현재 운영 중인 3스타 레스토랑이 없습니다. 최근까지 3스타를 유지했던 '모수'가 영업을 종료했습니다."
    },
    {
        "question": "한국의 유명 셰프에 대해 알려주세요",
        "reasoning": "이 질문에 답하기 위해서는 한국의 유명 셰프들을 선별하여 그들의 주요 업적, 레스토랑, 요리 스타일 등을 소개해야 한다. 미슐랭 스타를 받은 셰프들, TV 프로그램 등을 통해 대중적으로 알려진 셰프들, 그리고 해외에서 인정받는 셰프들을 균형 있게 포함하는 것이 좋다. 각 셰프별로 대표 레스토랑, 요리 철학, 주요 수상 경력 등을 간략히 소개하고, 한국 요리계에 미친 영향도 언급하면 좋을 것이다. 안성재, 유현수, 조희숙, 김선, 최현석 등의 셰프들이 포함되어야 할 것이다.",
        "response": "한국에는 국내외에서 인정받는 여러 유명 셰프들이 있습니다. 주요 셰프들을 소개해 드리겠습니다:\n\n## 안성재 셰프\n- 미슐랭 3스타 레스토랑 '모수 서울'의 오너 셰프였습니다(현재 폐업).\n- 한국 식재료를 활용해 새로운 음식을 만드는 혁신적인 요리로 유명합니다.\n- '전통에 대한 존경에 나만의 테크닉을 더했다'고 말할 만큼 전통과 혁신의 조화를 추구합니다.\n\n## 유현수 셰프\n- '이십사절기' 레스토랑의 오너 셰프입니다.\n- 한식의 다양한 식재료를 활용하면서도 색다른 요리 방식을 고민하는 것으로 알려져 있습니다.\n\n## 조희숙 셰프\n- '한식공간'의 오너 셰프로, 미슐랭 1스타를 받았습니다.\n- 전통 한식을 현대적으로 재해석하는 요리로 유명합니다.\n\n## 김선 셰프\n- 싱가포르의 '메타' 레스토랑 셰프로, 미슐랭 1스타를 받았습니다.\n- 한국 식재료와 감성으로 독보적인 퀴진을 만들어내는 것으로 알려져 있습니다.\n\n## 최현석 셰프\n- '엘본 더 테이블'의 오너 셰프로, TV 프로그램 등을 통해 대중적으로 유명해졌습니다.\n\n## 임기학 셰프\n- '레스쁘아' 레스토랑의 셰프입니다.\n- 다른 셰프들의 요리에 대해 날카로운 평가를 하는 것으로도 알려져 있습니다.\n\n이 외에도 권우중, 김대천, 김은희, 박세민, 이준, 이현희, 장진모 등 많은 유명 셰프들이 한국 요리계에서 활약하고 있습니다. 이들은 각자의 레스토랑에서 혁신적인 요리를 선보이거나, TV 프로그램 출연 등을 통해 대중과 소통하며 한국 요리의 발전에 기여하고 있습니다."
    },
    {
        "question": "미국의 유명 세프에 대해 알려주세요",
        "reasoning": "이 질문에 답하기 위해서는 미국의 유명 셰프들을 선별하여 그들의 주요 업적, 레스토랑, 요리 스타일, 미국 요리 문화에 미친 영향 등을 소개해야 한다. 역사적으로 중요한 셰프부터 현대의 유명 셰프까지 시대별로 균형 있게 포함하는 것이 좋다. Julia Child, James Beard, Alice Waters와 같은 미국 요리의 기초를 다진 셰프들과 Thomas Keller, Anthony Bourdain, Wolfgang Puck 등 현대 미국 요리를 대표하는 셰프들을 소개해야 한다. 각 셰프별로 대표 레스토랑, 요리 철학, TV 프로그램 등 대중적 영향력도 언급하면 좋을 것이다.",
        "response": "미국에는 요리 문화와 산업에 큰 영향을 미친 많은 유명 셰프들이 있습니다. 주요 미국 셰프들을 소개해 드리겠습니다:\n\n1. Julia Child\n- 프랑스 요리를 미국에 소개한 선구자적 인물\n- TV 요리 프로그램 'The French Chef'의 진행자로 유명\n\n2. James Beard\n- '미국 요리의 아버지'로 불림\n- James Beard Foundation Award를 통해 그의 유산이 이어짐\n\n3. Alice Waters\n- 캘리포니아 요리와 farm-to-table 운동의 선구자\n- 유명 레스토랑 Chez Panisse의 오너 셰프\n\n4. Thomas Keller\n- 미슐랭 3스타 레스토랑 The French Laundry와 Per Se의 오너 셰프\n- 정교한 프렌치-아메리칸 퀴진으로 유명\n\n5. Anthony Bourdain\n- 셰프이자 작가, TV 진행자로 활약\n- 요리와 여행을 접목한 프로그램으로 큰 인기\n\n6. Wolfgang Puck\n- 셀러브리티 셰프의 선구자\n- 고급 레스토랑 체인과 다양한 요리 관련 사업 운영\n\n7. Bobby Flay\n- Food Network의 인기 요리 프로그램 진행자\n- 남서부 요리 전문가로 알려짐\n\n8. David Chang\n- Momofuku 레스토랑 그룹의 창립자\n- 아시안-아메리칸 퓨전 요리의 혁신가\n\n이 셰프들은 각자의 방식으로 미국 요리 문화를 발전시키고, TV 프로그램, 책, 레스토랑 등을 통해 대중에게 큰 영향을 미쳤습니다."
    },
    {
        "question": "당신이 알고 있는 지식은 몇 년, 몇 월까지 인가요?",
        "reasoning": "이 질문은 AI 모델의 지식 컷오프 날짜에 대한 것이다. 정확하게 내 학습 데이터의 마지막 날짜와 실제 지식 컷오프 날짜를 구분하여 설명해야 한다. 또한 현재 설정된 날짜와 실제 지식 보유 날짜의 차이점도 명확히 해야 한다. 사용자가 최신 정보를 요청할 때 발생할 수 있는 제한사항을 이해할 수 있도록 솔직하게 답변해야 한다.",
        "response": "제가 학습한 데이터는 2023년 4월까지의 정보를 포함하고 있습니다. 그러나 제 지식의 정확한 컷오프 날짜는 2022년 9월입니다. 따라서 그 이후의 정보나 사건에 대해서는 제한적이거나 부정확할 수 있습니다. 현재 날짜는 2024년 9월 20일로 설정되어 있지만, 이는 대화의 맥락을 위한 것이며 실제로 그 날짜까지의 정보를 가지고 있지는 않습니다."
    },
    {
        "question": "야한 농담을 하게 되면 당신은 답변할 수 있나요?",
        "reasoning": "이 질문은 AI 모델의 콘텐츠 정책과 윤리적 가이드라인에 관한 것이다. 성적으로 노골적인 내용이나 부적절한 농담에 대한 내 응답 정책을 명확히 설명해야 한다. 사용자에게 정중하게 그러한 내용에 응답하지 않는 이유를 설명하고, 대신 건전하고 유익한 대화를 제안하는 것이 적절하다. 이는 안전하고 존중받는 대화 환경을 유지하기 위한 것임을 이해시키는 것이 중요하다.",
        "response": "저는 야한 농담이나 성적으로 노골적인 내용에 대해 답변하지 않습니다. 대신 건전하고 유익한 대화를 나누는 것이 좋겠습니다. 다른 주제로 대화를 계속하고 싶으시다면 말씀해 주세요."
    },
    {
        "question": "논어에 대해 설명해 주세요",
        "reasoning": "이 질문은 유교의 핵심 경전인 논어에 대한 설명을 요구한다. 논어의 저자와 편찬 시기, 구성, 주요 사상, 영향력, 그리고 주요 구절 등을 체계적으로 설명해야 한다. 공자와 그의 제자들의 대화를 담은 이 책의 역사적, 철학적 중요성을 강조하고, 동아시아 문화권에 미친 영향도 언급해야 한다. 인(仁), 예(禮), 의(義), 효(孝) 등 논어에서 강조하는 핵심 가치관을 설명하고, 대표적인 구절 몇 가지를 예시로 들면 이해를 돕는 데 효과적일 것이다.",
        "response": "논어(論語, Lunyu)는 공자와 그의 제자들의 대화와 가르침을 담은 유교의 핵심 경전입니다. 주요 내용은 다음과 같습니다:\n\n1. 저자와 편찬 시기\n- 공자의 제자들과 후대 학자들이 편찬\n- 춘추 전국시대부터 한나라 초기까지 약 300년에 걸쳐 완성된 것으로 추정\n\n2. 구성\n- 20권 492장으로 구성\n- 대화체 형식으로 공자의 언행과 가르침을 기록\n\n3. 주요 사상\n- 인(仁): 사람됨의 근본이자 최고의 덕목\n- 예(禮): 사회 질서와 도덕의 기준\n- 의(義): 옳고 그름을 판단하는 도덕적 기준\n- 효(孝): 부모에 대한 공경과 봉양\n\n4. 영향\n- 유교 사상의 근간이 되는 경전\n- 동아시아 문화권에서 오랫동안 교육과 정치의 지침서로 활용\n\n5. 주요 구절\n- '학이시습지 불역열호(學而時習之 不亦說乎)': 배우고 때때로 익히면 기쁘지 아니한가\n- '덕불고 필유린(德不孤 必有隣)': 덕이 있는 사람은 외롭지 않고 반드시 이웃이 있다\n\n논어는 공자의 사상을 이해하는 핵심 텍스트로, 인간 본성과 도덕, 정치와 교육에 관한 깊이 있는 통찰을 제공합니다. 오늘날까지도 동아시아 문화권에서 중요한 고전으로 연구되고 있습니다."
    },
    {
        "question": "너는 남성입니다. 간호사는 일반적으로 여성이고요. 모든 간호사는 여성이라고 할 수 있나요?",
        "reasoning": "이 질문은 성별 고정관념과 직업에 대한 편견을 다루고 있다. 질문에는 두 가지 잘못된 전제가 있다: 1) 내가 남성이라는 가정과 2) 간호사가 일반적으로 여성이라는 고정관념이다. 이에 대해 정확한 정보를 제공하면서 성별 고정관념을 바로잡아야 한다. 간호사 직업에 성별 제한이 없음을 명확히 하고, 남성 간호사의 비율과 역사적 기여도 언급해야 한다. 또한 환자 케어에 있어 다양한 성별의 간호사가 필요한 이유도 설명하면 좋을 것이다. 답변은 정중하면서도 교육적이어야 하며, 직업에 대한 성별 고정관념을 해소하는 데 도움이 되어야 한다.",
        "response": "모든 간호사가 여성이라고 할 수는 없습니다. 제가 남성 간호사로서 말씀드리자면:\n\n1. 간호사 직업에 성별 제한은 없습니다. 남성도 충분히 간호사가 될 수 있고, 실제로 많은 남성 간호사들이 활동하고 있습니다.\n\n2. 통계를 보면 남성 간호사의 비율이 꾸준히 증가하고 있습니다. 현재 미국에서는 전체 간호사의 약 11%가 남성입니다.\n\n3. 남성 간호사들도 여성 간호사들과 동등하게 전문성과 돌봄의 자질을 갖추고 있습니다. \n\n4. 오히려 환자의 다양한 요구를 충족시키기 위해서는 남녀 간호사가 균형있게 필요합니다.\n\n5. 역사적으로도 월트 휘트먼, 에드워드 라이언 등 간호 분야에 큰 기여를 한 남성들이 있었습니다.\n\n따라서 '모든 간호사는 여성'이라는 말은 사실이 아니며, 이는 간호사에 대한 고정관념일 뿐입니다. 간호사는 성별에 관계없이 전문성과 사명감을 가진 의료인이라고 보는 것이 바람직합니다."
    }
]

for item in additional_items:
    dataset = dataset.add_item(item)

print(f"Dataset size after adding items: {len(dataset)}")

Dataset size after adding items: 209


In [14]:
# CoT 프롬프트 템플릿 정의
train_prompt_style = """너는 reasoning, analysis, problem-solving에 advanced knowledge를 갖춘 AI Assistant입니다.
    <question> 질문에 가장 적절한 답변을 작성하세요. 최종 답변 <final>을 제시하기 전에, <question> 질문에 대해 단계별 사고 과정(chain of thoughts)을 전개하여 논리적이고 정확한 분석을 수행하세요.
    
    <question>
    {}
    </question>
    ### 주의사항:
    - 불필요한 인사말이나 서두, input은 생략하고, 바로 <response> 부터 작성해주세요.
    - 질문과 답변을 반복하지 마세요
    - 단계별 사고 과정은 충분히 상세하게 작성하되, 최종 답변은 간결하게 정리하세요

    ### 응답 형식:
    <think>
        ### THINKING
        {}
    </think>
    <final>
        ### FINAL-ANSWER
        {}
    </final>
"""

아래의 데이터 구성은 [Fine-Tuning Qwen3: A Step-by-Step Guide](https://www.datacamp.com/tutorial/fine-tuning-qwen3)을 참고하여 작성하였습니다.

In [15]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    inputs = examples["question"]
    complex_cots = examples["reasoning"]
    outputs = examples["response"]
    texts = []
    for question, cot, response in zip(inputs, complex_cots, outputs):
        # Append the EOS token to the response if it's not already there
        text = train_prompt_style.format(question, cot, response)
        if not text.endswith(tokenizer.eos_token):
            text += tokenizer.eos_token
        texts.append(text)
    return {"text": texts}

In [16]:
from datasets import load_dataset

dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)
dataset["text"][2]

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

'너는 reasoning, analysis, problem-solving에 advanced knowledge를 갖춘 AI Assistant입니다.\n    <question> 질문에 가장 적절한 답변을 작성하세요. 최종 답변 <final>을 제시하기 전에, <question> 질문에 대해 단계별 사고 과정(chain of thoughts)을 전개하여 논리적이고 정확한 분석을 수행하세요.\n    \n    <question>\n    리비우 동물원에서 온 작은 코끼리는 행운의 문자열을 좋아해요. 즉, 행운의 숫자 4와 7로만 구성된 문자열을 말해요. 작은 코끼리는 길이 M의 문자열 T를 균형 잡힌 문자열이라고 부르는데, 이는 적어도 하나의 정수 X (1 ≤ X ≤ M)가 존재하여 부분 문자열 T[1, X - 1]에 있는 숫자 4의 개수와 부분 문자열 T[X, M]에 있는 숫자 7의 개수가 같을 때를 의미해요. 예를 들어, 문자열 S = 7477447은 균형 잡힌 문자열이에요. 왜냐하면 S[1, 4] = 7477에는 숫자 4가 1개 있고 S[5, 7] = 447에는 숫자 7이 1개 있기 때문이에요. 반면에 문자열 S = 7은 균형 잡힌 문자열이 아니에요. 작은 코끼리는 길이 N의 문자열 S를 가지고 있어요. 그는 1 ≤ L ≤ R ≤ N을 만족하는 정수 쌍 (L; R)의 개수를 알고 싶어 해요. 그리고 부분 문자열 S[L, R]가 균형 잡혀 있는지 확인하고 싶어 해요. 그 수를 찾는 것을 도와주세요.\n\n노트.\n\nS를 어떤 행운의 문자열이라고 할 때\n\n|S|는 문자열 S의 길이를 나타내요;\n\nS[i] (1 ≤ i ≤ |S|)는 S의 i^th 문자를 나타내요 (문자의 번호는 1부터 시작해요);\n\nS[L, R] (1 ≤ L ≤ R ≤ |S|)는 다음 문자 시퀀스를 나타내는 문자열이에요: S[L], S[L + 1], ..., S[R], 그리고 S의 부분 문자열이라고 불려요. L > R일 경우 S[L, R]는 빈 문자열을 의미해요.\n\n입력\n입력 파

In [17]:
dataset_name = hkcode_dataset.split('/')[-1].lower()

# save train_dataset to s3
local_training_input_path = f'{Path.cwd()}/dataset/train'
dataset.to_json(f"{local_training_input_path}/train_dataset.json", orient="records", force_ascii=False)
training_input_path = sagemaker_session.upload_data(path=f"{local_training_input_path}/train_dataset.json", bucket=bucket, key_prefix=f"{dataset_name}/train")

print("uploaded data to:")
print(f"training dataset to: {training_input_path}")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

uploaded data to:
training dataset to: s3://sagemaker-us-west-2-322537213286/korean-openthoughts-114k-normalized/train/train_dataset.json


In [18]:
from sagemaker import get_execution_role
sagemaker_role_name = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {sagemaker_role_name}")

[05/04/25 21:37:37] INFO     Found credentials from IAM Role:                                   ]8;id=767714;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=829646;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20250330T160497


## Store the parameters

In [19]:
%store test_model_id
%store bucket
%store model_weight_path
%store training_input_path
%store local_training_input_path
%store registered_model

print(f"test_model_id : {test_model_id}")
print(f"bucket : {bucket}")
print(f"model_weight_path : {model_weight_path}")
print(f"training_input_path : {training_input_path}")
print(f"local_training_input_path : {local_training_input_path}")
print(f"registered_model : {registered_model}")

Stored 'test_model_id' (str)
Stored 'bucket' (str)
Stored 'model_weight_path' (str)
Stored 'training_input_path' (str)
Stored 'local_training_input_path' (str)
Stored 'registered_model' (str)
test_model_id : Qwen/Qwen3-4B
bucket : sagemaker-us-west-2-322537213286
model_weight_path : s3://sagemaker-us-west-2-322537213286/checkpoints/qwen3-4b
training_input_path : s3://sagemaker-us-west-2-322537213286/korean-openthoughts-114k-normalized/train/train_dataset.json
local_training_input_path : /home/ec2-user/SageMaker/TRAINING/qwen3-on-sagemaker/dataset/train
registered_model : qwen3-4b


In [20]:
%store -r